In [1]:
import pandas as pd
import os

import numpy as np

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

# 1. Скачайте данные по ссылке
https://www.kaggle.com/competitions/titanic/data?select=train.csv 

In [2]:
# имя файла с данными
data_file_dir = u"./data"
data_file_names = [os.path.join(data_file_dir, x) for x in os.listdir(data_file_dir)]
data_file_names

['./data/test.csv', './data/train.csv', './data/gender_submission.csv']

In [3]:
# данные и их структура
print("==Данные и их структура из файла train")
data = pd.read_csv(data_file_names[1])
print(data.shape)
print(data.info())
data.sample(7)

==Данные и их структура из файла train
(891, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
419,420,0,3,"Van Impe, Miss. Catharina",female,10.0,0,2,345773,24.1500,NaN,S
484,485,1,1,"Bishop, Mr. Dickinson H",male,25.0,1,0,11967,91.0792,B49,C
204,205,1,3,"Cohen, Mr. Gurshon ""Gus""",male,18.0,0,0,A/5 3540,8.0500,NaN,S
349,350,0,3,"Dimic, Mr. Jovan",male,42.0,0,0,315088,8.6625,NaN,S
391,392,1,3,"Jansson, Mr. Carl Olof",male,21.0,0,0,350034,7.7958,NaN,S
791,792,0,2,"Gaskell, Mr. Alfred",male,16.0,0,0,239865,26.0000,NaN,S
127,128,1,3,"Madsen, Mr. Fridtjof Arne",male,24.0,0,0,C 17369,7.1417,NaN,S


In [61]:
# количество уникальных значений по каждому из признаков
print(data.shape)
data.nunique()

(891, 12)


PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64

In [5]:
# баланс целевого признака 
data['Survived'].value_counts(normalize=True)

0    0.616162
1    0.383838
Name: Survived, dtype: float64

# 2. Исключите признаки, которые на ваш взгляд, могут привести к переобучению 
(например, id).

In [7]:
print("==Исходный набор признаков:")
features = data.columns
list(features)

==Исходный набор признаков:


['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [63]:
print("==Набор признаков для обучения:")
# 
good_features = \
[\
 # 'PassengerId',
 # 'Survived', # Выживание 0 = Нет, 1 = Да
 'Pclass', # Класс билета 1 = 1-й, 2 = 2-й, 3 = 3-й
 # 'Name',  # имя
 'Sex',    # пол
 'Age',    # возраст в годах
 'SibSp',   # Количество братьев и сестер / супругов на борту "Титаника"
 'Parch',   # Количество родителей / детей на борту "Титаника"
 # 'Ticket', # номер билета
 'Fare',  # тариф
 'Cabin', # номер каюты
 'Embarked' # Порт посадки C = Шербур, Q = Квинстаун, S = Саутгемптон
]

good_features

==Набор признаков для обучения:


['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked']

# 3. Разделите выборку на train и test.

In [17]:
X = data[good_features]
y = data['Survived']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y,  random_state=11)

# 4. Преобразуйте признаки
 категориальные признаки с помощью sklearn.preprocessing.OneHotEncoder, 
 
 а некатегориальные с помощью sklearn.preprocessing.StandardScaler (на следующем занятии расскажем, зачем это нужно)

In [58]:
X_train.sample(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
417,2,female,18.0,0,2,13.0000,NaN,S
550,1,male,17.0,0,2,110.8833,C70,C
846,3,male,NaN,8,2,69.5500,NaN,S
795,2,male,39.0,0,0,13.0000,NaN,S
820,1,female,52.0,1,1,93.5000,B69,S


In [60]:
# категориальные признаки с помощью sklearn.preprocessing.OneHotEncoder,
ohe = OneHotEncoder()
z=ohe.fit_transform(X_train[['Sex']]).toarray()
X_train["OHE_SEX"] = list(z)
X_train.sample(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,OHE_SEX
866,2,female,27.0,1,0,13.8583,NaN,C,"[1.0, 0.0]"
574,3,male,16.0,0,0,8.0500,NaN,S,"[0.0, 1.0]"
182,3,male,9.0,4,2,31.3875,NaN,S,"[0.0, 1.0]"
391,3,male,21.0,0,0,7.7958,NaN,S,"[0.0, 1.0]"
884,3,male,25.0,0,0,7.0500,NaN,S,"[0.0, 1.0]"


In [39]:
X_train[['Sex']]

,Sex
731,male
820,female
272,female
420,male
214,male
...,...
269,female
337,female
91,male
80,male


In [31]:
X_train['Sex']


731      male
820    female
272    female
420      male
214      male
        ...  
269    female
337    female
91       male
80       male
703      male
Name: Sex, Length: 668, dtype: object